In [1]:
# importing required modules
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import LineTokenizer
import glob
import re
import os
from typing import Dict, Any, AnyStr
import uuid
import pandas as pd
import docx2txt
import textract
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import string
from docx.api import Document
# from tabula import read_pdf
# from tabulate import tabulate
import numpy as np
# from fastapi import FastAPI, HTTPException
from transformers import BertModel, BertTokenizer
import csv

from sentence_transformers import SentenceTransformer, util

#pandas max columns and rows
pd.set_option('display.max_colwidth', None)
pd.set_option("display.max_rows", None)

RuntimeError: Failed to import transformers.models.bert.modeling_bert because of the following error (look up to see its traceback):
[WinError 182] The operating system cannot run %1. Error loading "C:\Users\me1awq\.conda\envs\cpu\lib\site-packages\torch\lib\shm.dll" or one of its dependencies.

In [ ]:
# Read the csv file
df = pd.read_csv(r"C:\Users\Omer\Desktop\SICK_train2.csv", usecols=[1, 2])

# Create a new DataFrame with the desired structure
new_df = pd.DataFrame(columns=["Document name", "Document text"])

# Loop through each column in the original DataFrame and append its data to the new DataFrame
for col in df.columns:
    name = col  # Name of the original column will become the Document name
    text = df[col].tolist()  # Data from the column will become the Document text
    new_df = new_df.append({"Document name": name, "Document text": text}, ignore_index=True)

# Print the new DataFrame
new_df



In [ ]:
# importing the stopwords using nltk library.
from nltk.corpus import stopwords
stop = stopwords.words('english')

custom_stopwords = ["ï","»","¿","â€“","â€™","a)","â","¿","–","–","b)","c)","d)","e)",":","(",")","â€˜","-",'must','used','using'
                   'near']
# punctation=[":","(",")"]
stop.extend(custom_stopwords)
# stop.extend(punctation)


new_df['Clean_documents_rules'] = new_df['Document text'].astype(str).str.lower()
#new_df['Clean_documents_rules']= new_df['Clean_documents_rules'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
new_df

In [ ]:
def token_sent(text):   
    sent_tokens=LineTokenizer(blanklines='keep').tokenize(text)
    return sent_tokens

In [ ]:
new_df['Sentence_Tokenize_rules']=new_df['Clean_documents_rules'].apply(token_sent) 
new_df['Word_Tokenize_rules']=new_df['Clean_documents_rules'].apply(word_tokenize) 
new_df['Sentence_Tokenize_rules'] = new_df['Sentence_Tokenize_rules'].apply(lambda x: [sent.strip() for sent in x])
new_df

In [ ]:
# Create a dictionary that maps document names to sentences
token_doc_name = dict(zip(new_df["Document name"], new_df["Sentence_Tokenize_rules"]))
token_doc_name

In [ ]:
#Create an embedding for all the sentences in the documents
from sentence_transformers import SentenceTransformer

# all the embedding used for semantic search

#model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')
#model = SentenceTransformer('ddobokki/electra-small-nli-sts')

#For Semantic textual similarity for small search queries
#model = SentenceTransformer('sentence-transformers/msmarco-distilroberta-base-v2')
model = SentenceTransformer('sentence-transformers/stsb-distilbert-base')
# sentence_embeddings = model.encode(sent_tokens)



In [ ]:
# finding the cosine similarity
def cosine_sim(embeddings1,embeddings2):
    """Cosine similarity metric function to calculate the distance between the two vectors."""
    cossim=( np.dot(embeddings1,embeddings2) )/ (np.linalg.norm(embeddings1)*np.linalg.norm(embeddings2))
    if np.isnan(np.sum(cossim)):
        return 0
    return cossim

In [ ]:
from itertools import chain
docs_sent_tokens=list(chain.from_iterable(new_df['Sentence_Tokenize_rules']))
#docs_name=main_df['Document name']
docs_sent_tokens

In [ ]:
# app = FastAPI()

output_file = 'search_results_SICK.csv'

@app.get("/vectorize")
async def vectorize(search_sentence: str):
    # Tokenize the sentence
    search_sentence_tokens = search_sentence.split(',')
    search_sentence_embeddings = [model.encode(sent) for sent in search_sentence_tokens]
    results=[]

    #set the threshold value to get the similarity result accordingly
    threshold=0.5

    #embedding all the documents and find the similarity between search text and all the tokenize sentences
    for doc_name, docs_sent_tokens in token_doc_name.items():
        doc_name = doc_name.title()
        for docs_sent_token in docs_sent_tokens:
            for search_sentence_embedding, search_sentence_token in zip(search_sentence_embeddings, search_sentence_tokens):
                sentence_embeddings = model.encode(docs_sent_token)
                sim_score1 = cosine_sim(search_sentence_embedding, sentence_embeddings)
                if sim_score1 > threshold:
                    results.append((
                    search_sentence_token.strip(),
                    docs_sent_token.strip(),
                    sim_score1,
                    search_sentence,
                    doc_name
                    ))
    #printing the top 10 matching result in dataframe format
    df=pd.DataFrame(results, columns=['Search Sentence', 'Matching Sentence','Similarity Score', 'Query Name', 'Document name'])

    # sorting in descending order based on the similarity score
    df.sort_values("Similarity Score", ascending = False, inplace = True)

    #change the value of n to see more results
    df_top10 = df.head(n=10)

    # Create a unique filename using a UUID
    filename = 'search_results_SICK.csv'

    # Write the top 10 results to a CSV file
    df_top10.to_csv(filename, index=False)

    # Read the CSV file and return its contents as a dictionary
    with open(filename, 'r') as f:
        result = csv.DictReader(f)
        output = [row for row in result]

    return {"data": output}


In [ ]:
import nest_asyncio
import uvicorn
if __name__ == "__main__":
    nest_asyncio.apply()
    uvicorn.run(app, host="0.0.0.0", port=8000)